In [1]:
import keras
import keras.backend as K
import tensorflow as tf 
import seaborn as sns
import cv2
import numpy as np 
import pandas as pd
from models import matting_net
from data_generator import *
from metrics import * 
import matplotlib.pyplot as plt

# the metric
def iou_coef(y_true, y_pred, smooth=1):
    
    threshold = tf.constant(0.5, dtype=tf.float64)
    
    y_true = tf.cast(y_true > threshold, dtype=tf.float64)
    y_pred = tf.cast(y_pred > threshold, dtype=tf.float64)
    
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3]) + K.sum(y_pred,[1,2,3]) - intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    
    return iou

Using TensorFlow backend.
/Users/hyunkim/.pyenv/versions/anaconda3-5.3.0/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/hyunkim/.pyenv/versions/anaconda3-5.3.0/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/hyunkim/.pyenv/versions/anaconda3-5.3.0/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16",

In [2]:
# model path for using in EDA
MODEL_PATH = "./trained_models/mattingnet/withBlankAndCustom/849-0.05.h5"
IMG_PATHS = "./dataset/img_paths.npy"

In [3]:
img_paths = np.load(IMG_PATHS)
img_paths[:2]

array(['./dataset/selfie/testing/00001.png',
       './dataset/selfie/testing/00002.png'], dtype='<U35')

In [4]:
# test and store ious
# model = matting_net((256,256,3), train=False)
model = matting_net((256,256,3), train=True)
model.load_weights(MODEL_PATH)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 8)  224         input_1[0][0]                    
__________________________________________________________________________________________________
separable_conv2d_1 (SeparableCo (None, 256, 256, 8)  144         conv2d_1[0][0]                   
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 256, 256, 8)  0           separable_conv2d_1[0][0]         
____________________________________________________________________________________________

In [5]:
model.output

[<tf.Tensor 'output/Sigmoid:0' shape=(?, 256, 256, 1) dtype=float32>,
 <tf.Tensor 'boundary_attention/Sigmoid:0' shape=(?, 256, 256, 1) dtype=float32>,
 <tf.Tensor 'refine/Mul_8:0' shape=(?, 256, 256, 1) dtype=float32>]

In [ ]:
import numpy as np 
import cv2
# import matplotlib.pyplot as plt

img = cv2.imread("./dataset/selfie/training/00694.png", cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (256,256))
# img = np.concatenate([img, np.ones((256,256,1))], axis=-1)[np.newaxis,:,:,:] / 255.
img = img.astype(np.float32)
img = img[np.newaxis,:,:,:] / 255.

mask = cv2.imread("./dataset/selfie/training/00694_matte.png", cv2.IMREAD_GRAYSCALE)
mask = cv2.resize(mask, (256,256))


pred = model.predict(img)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(19,7))
plt.subplot(141)
plt.imshow(img.squeeze(0)[:,:,:3])
plt.axis("off")

plt.subplot(142)
plt.imshow(mask)
plt.axis("off")

plt.subplot(143)
plt.imshow(pred.squeeze(0).squeeze(-1))
plt.axis("off")

# plt.subplot(144)
# plt.imshow(pred[1].squeeze(0).squeeze(-1))
# plt.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np 
import cv2
# import matplotlib.pyplot as plt

img_ = cv2.imread("./photo_2019-12-24 15.25.05.jpeg", cv2.IMREAD_COLOR)
img_ = cv2.cvtColor(img_, cv2.COLOR_BGR2RGB)
sap = img_.shape
img = cv2.resize(img_, (256,256))
img = img.astype(np.float32)
img = img[np.newaxis,:,:,:] / 255.

pred = model.predict(img)

In [ ]:
trues_ = tf.constant(img , dtype=tf.float64)
preds_ = tf.constant(pred[0], dtype=tf.float64) 

iou = iou_coef(trues_, preds_, smooth=1)

with tf.Session()  as sess :
    iou_ = sess.run(iou)
    print(iou_)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(19,7))
plt.subplot(121)
plt.imshow(img.squeeze(0)[:,:,:3])
plt.axis("off")

plt.subplot(122)
plt.imshow(pred.squeeze(0).squeeze(-1))
plt.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
sap[0]

In [ ]:
plt.figure(figsize=(19,7))
plt.subplot(121)
plt.imshow(img_[:,:,:3])
plt.axis("off")

plt.subplot(122)
plt.imshow(cv2.resize(pred[0].squeeze(0).squeeze(-1), (sap[1], sap[0])))
plt.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# data feeding logic
test_params = {
            'dim': (256,256),
            'batch_size': 1,
            'n_channels': 3,
            'shuffle': False,
            'augment': False,
            'train' : False,
        }

gen = DataGeneratorMatting(img_paths, **test_params)

In [ ]:
iou_list = []
for ge in gen:
    g, l = ge
    preds = model.predict(g, verbose=1)
    trues_ = tf.constant(l[0] , dtype=tf.float64)
    preds_ = tf.constant(preds, dtype=tf.float64) 

    iou = iou_coef(trues_, preds_, smooth=1)
    
    with tf.Session()  as sess :
        iou_ = sess.run(iou)
        print(iou_)
        iou_list.append(iou_)



In [ ]:
len(iou_list)

In [ ]:
sns.distplot(iou_list)